<a href="https://colab.research.google.com/github/zRiad001/SmartParkingSystem-ESP32/blob/main/Copy_of_Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.linear_model import LogisticRegression
import pandas as pd

In [ ]:
df = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')
print(df.info)

<bound method DataFrame.info of       customerID  gender  SeniorCitizen Partner Dependents  tenure  \
0     7590-VHVEG  Female              0     Yes         No       1   
1     5575-GNVDE    Male              0      No         No      34   
2     3668-QPYBK    Male              0      No         No       2   
3     7795-CFOCW    Male              0      No         No      45   
4     9237-HQITU  Female              0      No         No       2   
...          ...     ...            ...     ...        ...     ...   
7038  6840-RESVB    Male              0     Yes        Yes      24   
7039  2234-XADUH  Female              0     Yes        Yes      72   
7040  4801-JZAZL  Female              0     Yes        Yes      11   
7041  8361-LTMKD    Male              1     Yes         No       4   
7042  3186-AJIEK    Male              0      No         No      66   

     PhoneService     MultipleLines InternetService OnlineSecurity  ...  \
0              No  No phone service             DSL 

In [ ]:
print(df.isnull().sum())

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64


In this scenario I converted yes/no values into the binary values like 0 and 1. Because classification algorithms predict numerical values

In [ ]:
df['Churn'] =df['Churn'].map({'Yes':1,'No':0})
print(df.info)

<bound method DataFrame.info of       customerID  gender  SeniorCitizen Partner Dependents  tenure  \
0     7590-VHVEG  Female              0     Yes         No       1   
1     5575-GNVDE    Male              0      No         No      34   
2     3668-QPYBK    Male              0      No         No       2   
3     7795-CFOCW    Male              0      No         No      45   
4     9237-HQITU  Female              0      No         No       2   
...          ...     ...            ...     ...        ...     ...   
7038  6840-RESVB    Male              0     Yes        Yes      24   
7039  2234-XADUH  Female              0     Yes        Yes      72   
7040  4801-JZAZL  Female              0     Yes        Yes      11   
7041  8361-LTMKD    Male              1     Yes         No       4   
7042  3186-AJIEK    Male              0      No         No      66   

     PhoneService     MultipleLines InternetService OnlineSecurity  ...  \
0              No  No phone service             DSL 

Now applying one-hot encoding(converting categorical data into numeric form)

In [ ]:

categorical_cols = [col for col in df.columns if df[col].dtype == 'object' and col != 'customerID']
print("\nCategorical columns to encode:", categorical_cols)

df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

print(df_encoded.columns)

df_encoded.drop('customerID', axis=1, inplace=True)


Categorical columns to encode: ['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'TotalCharges']
Index(['customerID', 'SeniorCitizen', 'tenure', 'MonthlyCharges', 'Churn',
       'gender_Male', 'Partner_Yes', 'Dependents_Yes', 'PhoneService_Yes',
       'MultipleLines_No phone service',
       ...
       'TotalCharges_995.35', 'TotalCharges_996.45', 'TotalCharges_996.85',
       'TotalCharges_996.95', 'TotalCharges_997.65', 'TotalCharges_997.75',
       'TotalCharges_998.1', 'TotalCharges_999.45', 'TotalCharges_999.8',
       'TotalCharges_999.9'],
      dtype='object', length=6561)


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
numerical_cols = ['tenure', 'MonthlyCharges']
df_encoded[numerical_cols] = scaler.fit_transform(df_encoded[numerical_cols])
print(df_encoded.head)

<bound method NDFrame.head of       SeniorCitizen    tenure  MonthlyCharges  Churn  gender_Male  \
0                 0 -1.277445       -1.160323      0        False   
1                 0  0.066327       -0.259629      0         True   
2                 0 -1.236724       -0.362660      1         True   
3                 0  0.514251       -0.746535      0         True   
4                 0 -1.236724        0.197365      1        False   
...             ...       ...             ...    ...          ...   
7038              0 -0.340876        0.665992      0         True   
7039              0  1.613701        1.277533      0        False   
7040              0 -0.870241       -1.168632      0        False   
7041              1 -1.155283        0.320338      1         True   
7042              0  1.369379        1.358961      0         True   

      Partner_Yes  Dependents_Yes  PhoneService_Yes  \
0            True           False             False   
1           False           Fal

In [ ]:
from sklearn.model_selection import train_test_split
X = df_encoded.drop('Churn', axis = 1)
y = df_encoded['Churn']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)

In [ ]:
print(f"\nTraining set shape: {X_train.shape}")
print(f"Testing set shape: {X_test.shape}")
print(f"Training set churn distribution:\n{y_train.value_counts(normalize=True)}")
print(f"Testing set churn distribution:\n{y_test.value_counts(normalize=True)}")


Training set shape: (5634, 6559)
Testing set shape: (1409, 6559)
Training set churn distribution:
Churn
0    0.734647
1    0.265353
Name: proportion, dtype: float64
Testing set churn distribution:
Churn
0    0.734564
1    0.265436
Name: proportion, dtype: float64


In [ ]:
from sklearn.linear_model import LogisticRegression
logistic_model = LogisticRegression(solver = 'liblinear', random_state=42)
logistic_model.fit(X_train, y_train)

LogisticRegression(random_state=42, solver='liblinear')

In [ ]:
y_pred = logistic_model.predict(X_test)
y_pred_proba = logistic_model.predict_proba(X_test)
print(f"First 10 predictions: {y_pred[:10]}")
print(f"First 10 predicted probabilities (for Churn=1): {y_pred_proba[:10]}")

First 10 predictions: [0 1 0 0 0 1 0 0 0 0]
First 10 predicted probabilities (for Churn=1): [[0.96783443 0.03216557]
 [0.29620025 0.70379975]
 [0.93528677 0.06471323]
 [0.60503831 0.39496169]
 [0.97742063 0.02257937]
 [0.37461094 0.62538906]
 [0.5347658  0.4652342 ]
 [0.87788872 0.12211128]
 [0.99526577 0.00473423]
 [0.6132676  0.3867324 ]]


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

print("\n--- Model Evaluation ---")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"Precision: {precision_score(y_test, y_pred):.4f}")
print(f"Recall: {recall_score(y_test, y_pred):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred):.4f}")

auc_score = roc_auc_score(y_test, y_pred_proba[:, 1])
print(f"AUC: {auc_score:.4f}")

print(confusion_matrix(y_test, y_pred))


--- Model Evaluation ---
Accuracy: 0.7970
Precision: 0.6358
Recall: 0.5508
F1-Score: 0.5903
AUC: 0.8405

Confusion Matrix:
[[917 118]
 [168 206]]
